In [40]:
### for 80mpc box
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.interpolate as interp
import scipy as sp
import re



global1 =[]
global2 =[]
global3 = []
global4 = []
def unPackRawFile(raw_path):
    """
    - unpacks the Raw conditions file. Not used for the neural network.
    """
    y = np.loadtxt(raw_path, skiprows=6)
    distance = y[:,0]
    pec_vel = y[:,1]
    temp = y[:,2]
    HI_density = y[:,3]
    gas_density = y[:,4]
    gas_metallicity = y[:,5]
    
    return distance, pec_vel, temp, HI_density, gas_density, gas_metallicity
    
def unPackRawFlux(flux_path):
    """
    - unpacks the Raw conditions file. Not used for the neural network.
    """
    y2 = np.genfromtxt(flux_path, skip_header=1, delimiter=' , ')
    velocity = y2[:,0]
    flux = y2[:,1] 
    return velocity, flux
def hubble_flow_convert(velocity, a, omega_m, omega_lam):
    """
    - uses hubble flow to convert from velocity to distance
    """
    aH = a * 100 * (omega_m / a ** 3 + omega_lam)** 0.5
    return velocity/aH
def resample(distance, item, new_distance):
    """
    - interpolates the distances so that we can resample. useful because the velocity after converting using hubble flow doesn't have the same positions as the underlying properties.
    - creates a consistent distance scale (obviously these distances are messed up by peculiar velocities)
    """
    f = interp.interp1d(distance, item)
    new_item = f(new_distance)
    
    return new_item
def getDir(path_LOS,linenumber=7):
    """
    the direction of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x.split(', ')[1]).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr
def getPos(path_LOS,linenumber=7):
    """
    the start position of the LOS is given inside each file, (in the comments)
    this function parses the comments to get that information
    """
    f = open(path_LOS)
    x = f.readlines()[linenumber]
    answer = re.search('\(([^)]+)', x).group(1)
    arr = np.array(answer.split(','),dtype=float)
    return arr
def convertSphereToCart(theta, phi):
    "converts a unit vector in spherical to cartesian, needed for getGalaxies"
    return np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])

def get_galaxies(gals, path_LOS, number,Lbox = 80,aexp = 0.1452):
    """
    function can take in the path of the galaxy file (.res) and the line of sight. Number should match the LOS #
    """   
    # Parameter
    #Lbox = 80  # box size
    #aexp = 0.1452  # scale factor for the epoch
    OmegaM = 0.3036
    h100 = 0.6814    # normalized Hubble constant
    pos = np.array(getPos(path_LOS))  # origin of LOS
    pos = pos/512*80
    sphere_los = np.array(getDir(path_LOS))  # direction of LOS , np.sum(e3**2) should be = 1
    e3 = convertSphereToCart(sphere_los[0], sphere_los[1])
    Mvir = gals[0] # SFR
    xg = gals[1] #\
    yg = gals[2] # | positions of the galaxies in cMpc/h
    zg = gals[3] #/
    vx = gals[4] #\
    vy = gals[5] # | velocities of the galaxies in km/s
    vz = gals[6] #/
    
    out_arr = Box(Lbox, OmegaM, h100, aexp, Mvir, xg, yg, zg, vx, vy, vz, pos, e3)
    num_arr = number*np.ones(len(out_arr.T))
    return np.vstack([out_arr, num_arr])
def Box(Lbox, OmegaM, h100, aexp, Mvir, xg, yg, zg, vx, vy, vz, pos, e3):
    """
    Wraps LOS to the original box (0-80), helper function for get galaxies.
    """
    ##### find intersection between LOS and grid
    planeNormal = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
    if e3[0] > 0:
        planePoint_x = np.array([[80, 0, 0],[160, 0, 0],[240, 0, 0]])
        planePoint_x_inverse = np.array([0,0,0])
    else:
        planePoint_x = np.array([[0, 0, 0],[-80, 0, 0],[-120, 0, 0]])
        planePoint_x_inverse = np.array([80,0,0])
    if e3[1] > 0:
        planePoint_y = np.array([[0, 80, 0],[0, 160, 0],[0, 240, 0]])
        planePoint_y_inverse = np.array([0,0,0])
    else:
        planePoint_y = np.array([[0, 0, 0],[0, -80, 0],[0, -160, 0]])
        planePoint_y_inverse = np.array([0,80,0])
    if e3[2] > 0:
        planePoint_z = np.array([[0, 0, 80],[0, 0, 160],[0, 0, 240]])
        planePoint_z_inverse = np.array([0,0,0])
    else:
        planePoint_z = np.array([[0, 0, 0],[0, 0, -80],[0, 0, -160]])
        planePoint_z_inverse = np.array([0,0,80])
    
    planePoint = np.vstack((planePoint_x,planePoint_y,planePoint_z))
    intercept_pts = LinePlaneIntersection(planeNormal=planeNormal, planePoint=planePoint, losDirection=e3, losPoint=pos)
    intercept_pts[:,0][0:3] = planePoint_x[:,0]
    intercept_pts[:,1][3:6] = planePoint_y[:,1]
    intercept_pts[:,2][6:] = planePoint_z[:,2]
    planePoint_inverse = np.vstack((planePoint_x_inverse,planePoint_y_inverse,planePoint_z_inverse))
    intercept_pts_inverse = LinePlaneIntersection(planeNormal=planeNormal, planePoint=planePoint_inverse, losDirection=(-1)*e3, losPoint=pos)
    for i in range(3):
        intercept_pts_inverse[i,i] = planePoint_inverse[i,i]
    los_start_pts_inverse = np.where(np.logical_and(e3>0, intercept_pts_inverse == 0), 80, intercept_pts_inverse)
    distance_inverse = (-1)*np.sqrt(np.sum((intercept_pts_inverse-pos)**2,axis=1))
    distance = np.sqrt(np.sum((intercept_pts-pos)**2,axis=1))
    new_los_start_pts = np.vstack((pos,intercept_pts[distance<270]%80))
    new_los_start_pts = np.where(np.logical_and(e3<0, new_los_start_pts == 0), 80, new_los_start_pts)
    new_los_start_pts = np.vstack((new_los_start_pts,los_start_pts_inverse))
    x = xg
    y = yg
    z = zg
    new_x = np.tile(x, (len(new_los_start_pts), 1))
    new_y = np.tile(y, (len(new_los_start_pts), 1))
    new_z = np.tile(z, (len(new_los_start_pts), 1))
    w = e3[0]*(new_x-new_los_start_pts[:,0][:, np.newaxis])+\
        e3[1]*(new_y-new_los_start_pts[:,1][:, np.newaxis])+\
        e3[2]*(new_z-new_los_start_pts[:,2][:, np.newaxis])
    dx = new_x - new_los_start_pts[:,0][:, np.newaxis] - w*e3[0]
    dy = new_y - new_los_start_pts[:,1][:, np.newaxis] - w*e3[1]
    dz = new_z - new_los_start_pts[:,2][:, np.newaxis] - w*e3[2]
    new_x = np.where(dx>40,new_x-80,new_x)
    new_x = np.where(dx<-40,new_x+80,new_x)
    new_y = np.where(dy>40,new_y-80,new_y)
    new_y = np.where(dy<-40,new_y+80,new_y)
    new_z = np.where(dz>40,new_z-80,new_z)
    new_z = np.where(dz<-40,new_z+80,new_z)
    w = e3[0]*(new_x-new_los_start_pts[:,0][:, np.newaxis])+\
        e3[1]*(new_y-new_los_start_pts[:,1][:, np.newaxis])+\
        e3[2]*(new_z-new_los_start_pts[:,2][:, np.newaxis])
    dx = new_x - new_los_start_pts[:,0][:, np.newaxis] - w*e3[0]
    dy = new_y - new_los_start_pts[:,1][:, np.newaxis] - w*e3[1]
    dz = new_z - new_los_start_pts[:,2][:, np.newaxis] - w*e3[2]
    distance_all = np.concatenate((np.insert(distance[distance<270],0,0),distance_inverse))[:, np.newaxis]
    w += distance_all
    # distance from the galaxy to the LOS
    dr = np.sqrt(dx**2+dy**2+dz**2)
    # convert w from real to redshift space
    aH = 100*aexp*np.sqrt(OmegaM/aexp**3+(1-OmegaM))
    w += (e3[0]*vx+e3[1]*vy+e3[2]*vz)/aH
    # convert dr from comoving Mpc/h to proper Mpc
    dr *= (aexp/h100)
    Mvir = np.tile(Mvir, (len(new_los_start_pts), 1))
    # select galaxies
    sel1 = np.logical_and(w>5,w<195) # within first 200 cMpc/h
    sel2 = np.logical_and(dr<.1,Mvir>10**8)
    sel = np.logical_and(sel1,sel2)
    if(len(sel) > 0):
        Mvir1 = Mvir[sel]
        w1 = w[sel]
        w1 = np.round(w1,decimals=8)
        dr1 = dr[sel]
        dr1 = np.round(dr1,decimals=8)
        global1.append(dx[sel])
        global2.append(dy[sel])
        global3.append(dz[sel])
        global4.append(new_x[sel])
        #gives Mvir, distance along LOS, and distance away from LOS
        #why have unique
        return np.array([Mvir1,w1,dr1])
    else:
        return np.array([[],[],[]])
def LinePlaneIntersection(planeNormal, planePoint, losDirection, losPoint):
    """
    Finds the intersection between LOS and grid
    This function does a lot of floating point arithmetic
    so the intersection at x=0 may be at x=-1e16, for example
    I correct the result in Box() because intersection at 0 means the LOS starting pt is 80
    """
    ndotu = planeNormal.dot(losDirection)
    w = losPoint - planePoint
    si = np.array([-planeNormal[i].dot(w[i*len(w)//3:(i+1)*len(w)//3].T) / ndotu[i] for i in [0,1,2]])
    Psi = w + np.vstack(np.multiply.outer(si,losDirection)) + planePoint
    return Psi
def get_LOS_distance_zspace(path_LOS, aexp, OmegaM, new_distance):
    """
    Distance along LOS is in real space. Needs to use peculiar velocity in .raw file to convert to redshift space.
    new_distance is in real space (where data is sampled)
    """
    distance, pec_vel, temp, HI_density, gas_density, gas_metallicity = unPackRawFile(path_LOS)
    aH = 100*aexp*np.sqrt(OmegaM/aexp**3+(1-OmegaM))
    x_redshift_space = distance + pec_vel/aH
    f = interp.interp1d(distance, x_redshift_space)
    new_distance_redshift_space = f(new_distance)
    
    return new_distance_redshift_space


In [2]:
#galaxy Mvir, x,y,z, vx, vy, vz
gals = np.loadtxt('hprops.res', usecols=[0,5,6,7],unpack=1) 
# setting vx, vy, vz to be 0, so the position is in real space, to match the v=0 LOS
gals_vel = np.zeros((3,gals.shape[1]))
gals = np.concatenate((gals,gals_vel))

In [83]:
LOS_num = 10

los_gal_list = []
for i in np.arange(0,LOS_num):
        #pulling data from galaxies
    flux_path = 'los.00' + '{0:03}'.format(i) +'.raw'
    
    los_gal_data = get_galaxies(gals, flux_path, i, Lbox = 80,aexp = 0.1452)
      
    los_gal_list.append(los_gal_data)

In [84]:
los_gal_list[0][0]

array([8.993e+10, 1.508e+10, 9.402e+09, ..., 3.641e+08, 3.427e+08,
       2.856e+08])

In [85]:
halo =  np.loadtxt('hprops.res', usecols=[0,5,8],unpack=1) 


In [86]:
halo[2]

array([0.4979  , 0.3528  , 0.3438  , ..., 0.00911 , 0.01239 , 0.009934])

In [87]:
def lyman_identification():
    """
    function for identifying lls using neutral fraction as filter
    """
    neutral_fraction_limit = 1e-3
    data = []
    l_bases = []
    r_bases = []
    for i in range(10):
        raw_path = 'los.00' + '{0:03}'.format(i) +'.raw'
        distance, pec_vel, temp, HI_density, gas_density, gas_metallicity = unPackRawFile(raw_path)
        #changing the distance from chimp to cmeters
        distance_cm = distance *0.1474*3.086e24 /0.68
        temp_data =[]
        temp_left =[]
        temp_right = []
        for k in range(len(distance) -2):
            if(HI_density[k]/gas_density[k]<neutral_fraction_limit and HI_density[k+1]/gas_density[k+1]>neutral_fraction_limit):
                left_bases = k
                peak_fraction = HI_density[k]/gas_density[k]
                peak_HI = HI_density[k]
                peak_distance = distance[k]
                if( k+1 <len(distance)):
                    while(not(HI_density[k]/gas_density[k]>neutral_fraction_limit and HI_density[k+1]/gas_density[k+1]<neutral_fraction_limit) and k <len(distance)-1):
                        right_bases = k
                        if(k<len(distance)-1):
                            k+=1
                        if(HI_density[k]/gas_density[k]>peak_fraction):
                            peak_fraction = HI_density[k]/gas_density[k]
                            peak_HI = HI_density[k]
                            peak_distance = distance[k]
                    area_under = peak_HI *1/2*(distance_cm[right_bases]-distance_cm[left_bases])/0.937415
                    if(area_under >= (1.6*10**17)and area_under  <= (10**20)):
                        temp_data.append(peak_distance)
                        temp_left.append(distance[left_bases])
                        temp_right.append(distance[right_bases])
        data.append(temp_data)
        l_bases.append(temp_left)
        r_bases.append(temp_right)
    return data,l_bases,r_bases
lyman_limited_value,left_bases,right_bases = lyman_identification()

In [88]:
lyman_limited_value

[[],
 [87.84425, 113.82144, 118.64769],
 [1.65065, 9.67578, 107.77678, 152.64549, 180.40597],
 [],
 [91.80602, 91.9904, 98.3885, 139.49486, 142.48364],
 [199.37549],
 [112.06471],
 [30.50207],
 [83.89149],
 [37.43754, 57.28296, 64.52623]]

In [89]:
##data is:gives sfr, distance along LOS, and distance away from LOS
#I am just going to simply look at distance vs sfr right here, any thing within 2 chimp away form the lls is counted as a galaxy associated with it
def associated_sphere(all_distance,data,counts):
    """
    this function finds every galaxy associated with lls in a sphere radius. However, it might double count 
    """
    ##going through every los
    associated = []
    for i in range(counts):
        ##going through the lls in the los
        for k in all_distance[i]:
            ##calculating the distance of every galaxy's relationship with lls
            for z in range(len(data[i][1])):
                if(np.square((k-data[i][1][z])**2 + (data[i][2][z]/0.1452*0.68)**2) <.1):
                    ##mass, distance along, distance away, distance from lls.
                    associated.append([data[i][1][z],data[i][2][z],global4[i][z],data[i][0][z],data[i][1][z]])
    return associated

In [140]:
len(global1[2])
#the dx,dy,dz have different dimensions after the first two los..

1009

In [141]:
len(global4[2])

1009

In [139]:
len(los_gal_list[2][1])

792

In [105]:
associated_sphere_value = associated_sphere(lyman_limited_value,los_gal_list,5)

IndexError: index 797 is out of bounds for axis 0 with size 792

In [101]:
associated_sphere_value

[[88.19422237, 0.06438698, 25.09365, 2877000000.0, 88.19422237],
 [88.0805912, 0.03509942, 24.94672, 2085000000.0, 88.0805912],
 [88.12446114, 0.05261711, 25.02666, 835200000.0, 88.12446114],
 [88.19490719, 0.07025449, 24.63494, 778100000.0, 88.19490719],
 [88.21957797, 0.06943368, 24.67144, 656800000.0, 88.21957797],
 [88.07177158, 0.02641437, 24.91112, 606800000.0, 88.07177158],
 [87.8455278, 0.0029493, 24.68048, 528300000.0, 87.8455278],
 [87.63257228, 0.02426999, 24.66493, 371200000.0, 87.63257228],
 [113.53489729, 0.0466248, 38.43353, 22060000000.0, 113.53489729],
 [113.92799811, 0.0143503, 38.81847, 6332000000.0, 113.92799811],
 [113.72069613, 0.06984666, 38.50241, 4398000000.0, 113.72069613],
 [113.95403683, 0.02863693, 38.95048, 3662000000.0, 113.95403683],
 [113.87925, 0.01052576, 38.77889, 3255000000.0, 113.87925],
 [113.83268403, 0.01810193, 38.73241, 2920000000.0, 113.83268403],
 [114.12186745, 0.039522, 39.03516, 2720000000.0, 114.12186745],
 [113.5868173, 0.04725546, 38.4

In [99]:
halo_size = []
for i in associated_sphere_value:
    for k in range(len(halo[0])):
        if(i[3] == halo[0][k] and i[2] == halo[1][k]):
            halo_size.append([i[0],i[1],i[3],halo[2][k]])

In [103]:
len(halo_size)
#dx,dy,mass, halo size

45

In [104]:
final_galaxies = []
for i in halo_size:
    if(np.sqrt(i[0]**2+(i[1]/0.1452*0.68)**2) < i[3]):
        final_galaxies.append(i)
final_galaxies

[]

In [95]:
def within_lls(lyman_limited_value,galaxies_along,galaxies_away,counts,halo_size):
    """
    this function looks at if the galaxies' dark halo covers the lyman_limited.
    """
